In [37]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.special import softmax

from dense_layer import DenseLayer
from relu_layer import ReluLayer

In [20]:
iris = load_iris()
X, y = iris.data, iris.target

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, stratify=y, test_size=0.25)

In [38]:
def calculate_grads(X, y):
    probs = softmax(X)
    n = X.shape[0]
    probs[range(n), y] -= 1
    return probs / n

In [63]:
class Network:
    def __init__(self, input_features: int) -> None:
        self.layers = []
        self.layers.append(DenseLayer(input_features, 40))
        self.layers.append(ReluLayer())
        self.layers.append(DenseLayer(40, 20))
        self.layers.append(ReluLayer())
        self.layers.append(DenseLayer(20, 3))
        
    def forward(self, X: np.ndarray):
        activations = []
        entr = X
    
        for layer in self.layers:
            activations.append(layer.forward(entr))
            entr = activations[-1]
    
        return activations
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        acts = softmax(forward(X))
        return acts
    
    def fit_batch(self, X_batch: np.ndarray, y_batch: np.ndarray) -> None:
        activations = self.layers.forward(X_batch)
    
        grads = calculate_grads(activations[-1], y_batch)
    
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            grads = layer.backward(activations[i], grads)

In [83]:
def iterate_batches(X, y, batch_size) -> np.ndarray:
    div = int(X.shape[0] / batch_size)
    while X.shape[0] % div != 0:
        div += 1
        
    X_split = np.array(np.split(X, div))
    y_split = np.array(np.split(y, div))
    
    
    return X_split, y_split

In [70]:
network = Network(X_train.shape[1])

for X_batch, y_batch in iterate_batches(X_train, y_train, batch_size=16):
    network.fit_batch(X_batch, y_batch)
    

TypeError: iterate_batches() missing 2 required positional arguments: 'X' and 'y'